# Testing False Nearest Neighbors for the sensor sphere example

In [5]:
from __future__ import division
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.distance import squareform
import importlib
import scipy.spatial as sp

## Load in sphere code
sys.path.append('../data/simple_egs/')
import SphereExample as sphere
import ContinuousTimeSphereModel as ctsm
importlib.reload(ctsm)
importlib.reload(sphere)

## Load in d.n. analysis code
sys.path.append('../src/')
import graph_fns as gf
import persistence_fns as pf
import sliding_window_fns as sw
from ripser import ripser
from persim import plot_diagrams
from sklearn import manifold

In [2]:
## Setup Sensor Lifetimes
T = 15 # Period 

obsfn = lambda t, p: sphere.periodic_northsouth_modulated(t,p,T)
edge_wtsfn = lambda hull_obj: sphere.get_edge_wts(hull_obj, alpha = 1.0)

lambda1 = 50
lambda2 = lambda1

sensor_lifetimes = ctsm.get_sensor_lifetimes(2.5*T, lambda1, lambda2)

In [3]:
## Create the Dynamic Network
dynamic_network_samples = 200
ts = np.linspace(0,2*T,dynamic_network_samples)

(node_wts,edge_wts, allpoints) = ctsm.sample_dynamic_network(sensor_lifetimes, ts, obsfn = obsfn,
                                    edge_wtsfn = edge_wtsfn)

In [6]:
## Apply the analysis

# apply phi functions, and scale the weights
phi_node_wts, phi_edge_wts = gf.weight_fn(node_wts, edge_wts, lamda=1, phi=sphere.linear_phi_fn)

# construct the filtrations / simplicial complexes according to our construction
filtration_matrix = list(map(lambda n, e: pf.get_filtration(n, e), phi_node_wts, phi_edge_wts))

# summarize these filtrations using H_0 barcodes
barcodes = list(map(pf.get_rips_complex, filtration_matrix))

bn_dist_matrix = pf.get_bottleneck_dist_matrix(barcodes)

## Apply FNN

In [19]:
import fnn_fns as fnn
importlib.reload(fnn)

<module 'FalseNearestNeighbors' from '../shared_scripts/FalseNearestNeighbors.py'>

In [21]:
for threshold in np.linspace(0,2,10):
    num_false_nbrs = fnn.false_nearest_neighbors(bn_dist_matrix, 20, threshold)
    print(num_false_nbrs)

[ 199.  198.  197.  196.  195.  194.  193.  192.  191.  190.  189.  188.
  187.  186.  185.  184.  183.  182.  181.]
[ 132.  189.  191.  195.  194.  193.  193.  192.  191.  190.  189.  188.
  187.  186.  185.  184.  183.  182.  181.]
[ 132.  182.  187.  192.  182.  183.  180.  173.  170.  168.  166.  168.
  168.  167.  169.  167.  163.  161.  160.]
[ 129.  152.  140.  129.  127.  128.  132.  136.  139.  135.  138.  137.
  134.  135.  134.  133.  131.  128.  125.]
[ 123.   94.   73.   75.   80.   91.   93.   99.  101.  104.  104.  102.
   98.   95.   98.   91.   89.   91.   87.]
[ 103.   56.   39.   46.   47.   51.   51.   51.   60.   59.   62.   63.
   67.   62.   64.   61.   58.   55.   52.]
[ 70.  34.  18.  22.  22.  19.  21.  26.  27.  24.  30.  30.  29.  28.  29.
  29.  27.  27.  27.]
[ 48.  13.   8.   7.   8.   6.   9.   8.   9.   7.   7.   9.  10.   6.   8.
  14.  12.  15.  13.]
[ 37.   4.   2.   1.   0.   0.   0.   1.   0.   2.   1.   1.   0.   0.   0.
   2.   4.   2.   4.]
[ 23